In [1]:
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup
import datetime
from dataclasses import dataclass, field
from IPython.display import display, Markdown, HTML
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Callable, Dict, List

## Load data

In [34]:
# Go to https://ctcultra.com/result-2024/, lalu save page as HTML (page only)
def read_ctc(filename: str, table_id: str) -> pd.DataFrame:
    with open(filename) as f:
        doc = f.read()
    soup = BeautifulSoup(doc, 'html.parser')
    tabs = soup.find_all(id=table_id)
    assert len(tabs), f"{table_id} not found"
    
    tab = tabs[0]
    tbody = tab.find_all('tbody')[0]
    trs = tbody.find_all('tr')
    
    names = []
    genders = []
    times = []
    
    for tr in trs:
        tds = tr.find_all('td')
        assert len(tds)==9
        
        names.append(tds[1].get_text())
        genders.append(tds[2].get_text())
        times.append(tds[5].get_text())
    
    df = pd.DataFrame({'name': names, 'gender': genders, 'time': times})
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['gender'] = df['gender'].replace('FEMALE', 'f').replace('MALE', 'm')
    assert set(df['gender'].unique()) == {'f', 'm'}
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    
    return df

def read_scorenow(filename: str) -> pd.DataFrame:
    df = pd.read_excel(filename)
    df = df[ df['bib'].notnull() ]
    df = df[ ~df['time'].isin(['DNS', 'DQ', 'Not started', 'Unofficial Winner'])]
    df = df[['name', 'gender', 'time']]
    
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['gender'] = df['gender'].replace('Female', 'f').replace('Male', 'm')
    assert set(df['gender'].unique()) == {'f', 'm'}
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s.lower() not in ['cot', 'dnf', 'dq'] else pd.NaT )
    return df

# Untuk BTR, lihat di result table, lalu copy isi tabel satu per satu per halaman ke OO Calc
def read_btr(filename: str) -> pd.DataFrame:
    return read_scorenow(filename)

# Untuk Mantra, open result page, liat XHR requests, copy jsonnya
def read_mantra(filename: str) -> pd.DataFrame:
    with open(filename) as f:
        doc = json.load(f)
    
    results = doc['result']
    cols = list(results[0].keys())
    data = {col: [] for col in cols}
    for r in results:
        for c in cols:
            data[c].append(r[c])
            
    df = pd.DataFrame(data)
    df = df[['fullname', 'gender', 'age', 'time']]
    df = df.rename(columns={'fullname': 'name'})
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['gender'] = df['gender'].replace('Female', 'f').replace('Male', 'm')
    assert set(df['gender'].unique()) == {'f', 'm'}
    df['age'] = df['age'].fillna(0).astype(int)
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    df = df.reset_index(drop=True)
    return df

# Untuk Bromo Tengger, pakai result excel yg disediakan panitia
def read_bromotengger(filename: str) -> pd.DataFrame:
    df = pd.read_csv(filename)
    df = df[['fullname', 'gender', 'time']]
    df = df.rename(columns={'fullname': 'name'})
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['gender'] = df['gender'].replace('FEMALE', 'f').replace('MALE', 'm')
    assert set(df['gender'].unique()) == {'f', 'm'}
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    df = df.reset_index(drop=True)
    return df

# Read results in sixrace.id
def read_sixrace(filename: str):
    with open(filename) as f:
        doc = f.read()
    soup = BeautifulSoup(doc, 'html.parser')
    tabs = soup.find_all('table', id='example')
    assert len(tabs), f"table id=example not found"
    
    tab = tabs[0]
    tbody = tab.find_all('tbody')[0]
    trs = tbody.find_all('tr')
    
    names = []
    times = []
    statuses = []
    
    for tr in trs:
        tds = tr.find_all('td')
        assert len(tds)==4
        
        names.append(tds[2].get_text().strip())
        infos = tds[3].get_text().split()
        times.append(infos[0])
        statuses.append(infos[1])
    
    df = pd.DataFrame({'name': names, 'time': times, 'status': statuses})
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    
    df = df[ df['status'] != 'DNS' ].reset_index(drop=True)
    statuses = df['status'].unique()
    assert set(statuses).issubset({'Finished', 'DNF', 'DQ'}), f"Found unknown status: " + str(statuses)
    
    max_time = df['time'].max()
    df.loc[ df['status']=='DNF', 'time'] = pd.NaT
    df = df.reset_index(drop=True)
    return df

# Untuk MSR, go https://sixrace.id/result/view.php?r=r243, save as html page once race cat per file
def read_msr(filename: str):
    return read_sixrace(filename)

# Untuk MMDT, go to https://sixrace.id/result/view.php?r=r247, save has html one race cat per file
def read_mmdt(filename: str):
    return read_sixrace(filename)

# Untuk SLU, open result page, liat XHR requests, copy jsonnya
def read_slu(filename: str) -> pd.DataFrame:
    with open(filename) as f:
        results = json.load(f)
    
    datas = []
    for r in results:
        data = {
            'name': r['value']['name'].strip().title(),
            'gender': r['value']['city'],
            'time': r['value']['time'],
        }
        datas.append(data)
            
    df = pd.DataFrame(datas)
    df['gender'] = df['gender'].replace('Pria', 'm').replace('Wanita', 'f')
    assert set(df['gender'].unique()) == {'f', 'm'}, 'Found ' + str(set(df['gender'].unique()))
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    df = df.reset_index(drop=True)
    return df

# Copy table from ITRA race result, paste to OO Calc
def read_itra(filename: str) -> pd.DataFrame:
    df = pd.read_excel(filename)
    df = df[['name', 'time', 'age', 'gender']]
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['gender'] = df['gender'].apply(lambda s: s.strip().replace('H', 'm').replace('F', 'f'))
    assert set(df['gender'].unique()) == {'f', 'm'}, 'Found ' + str(set(df['gender'].unique()))
    df['time'] = df['time'].astype(str)
    df['time'] = df['time'].apply(lambda s: s.replace(' AM', '') if s else '')
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    df = df.reset_index(drop=True)
    return df

# Go to https://pickmyrace.id/events/dieng-caldera-race-2024/, lalu save page as HTML (page only)
def read_pickmyrace(filename: str, table_id: str) -> pd.DataFrame:
    with open(filename) as f:
        doc = f.read()
    soup = BeautifulSoup(doc, 'html.parser')
    tabs = soup.find_all(id=table_id)
    assert len(tabs), f"{table_id} not found"
    
    tab = tabs[0]
    tbody = tab.find_all('tbody')[0]
    trs = tbody.find_all('tr')
    
    names = []
    genders = []
    times = []
    statuses = []
    
    for tr in trs:
        tds = tr.find_all('td')
        assert len(tds)>=9
        
        names.append(tds[1].get_text().strip().title())
        genders.append(tds[2].get_text().strip())
        times.append(tds[8].get_text().strip())
        statuses.append(tds[9].get_text().strip())
    
    df = pd.DataFrame({'name': names, 'gender': genders, 'time': times, 'status': statuses })
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['gender'] = df['gender'].replace('FEMALE', 'f').replace('MALE', 'm')
    assert set(df['gender'].unique()).issubset({'f', 'm'}), 'Found ' + str(set(df['gender'].unique()))
    
    df = df[ ~df['status'].isin(['DNS', 'DQ']) ]
    assert set(df['status'].unique()).issubset({'FINISHER', 'DNF', 'COT'}), 'Found ' + str(set(df['status'].unique()))
    
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    
    df.loc[ df['time'] >= pd.Timedelta(hours=99), 'time'] = pd.NaT
    df = df.reset_index(drop=True)
    return df

# Go to https://pickmyrace.id/events/dieng-caldera-race-2024/, lalu save page as HTML (page only)
def read_dcr(filename: str, table_id: str) -> pd.DataFrame:
    return read_pickmyrace(filename, table_id)

# Go to https://bromomarathon.com/past-results, check XHR
def read_bromar(filename: str, category: str) -> pd.DataFrame:
    with open(filename) as f:
        results = json.load(f)
    
    datas = []
    for r in results:
        data = {
            'name': r['full_name'].strip().title(),
            'gender': r['gender'],
            'time': r['finish_time'],
            'category': r['category'],
            'status': r['status'],
            'country': r['nationality'],
        }
        datas.append(data)
            
    df = pd.DataFrame(datas)
    df['gender'] = df['gender'].replace('Female', 'f').replace('Male', 'm')
    assert set(df['gender'].unique()) == {'f', 'm'}, 'Found ' + str(set(df['gender'].unique()))
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s else pd.NaT )
    
    df = df[ df['category']==category ]
    df = df[ ~df['status'].isin(['DNS', 'DQF']) ]
    df = df.reset_index(drop=True)
    return df

# Untuk Rinjani100, lihat di result table, lalu copy isi tabel satu per satu per halaman ke OO Calc
def read_rinjani100(filename: str) -> pd.DataFrame:
    df = pd.read_excel(filename)
    df = df[ df['bib'].notnull() ]
    df = df[ ~df['time'].isin(['DNS', 'DQ'])]
    df = df[['name', 'gender', 'time']]
    
    df['name'] = df['name'].apply(lambda s: s.strip().title())
    df['gender'] = df['gender'].replace('Female', 'f').replace('Male', 'm')
    assert set(df['gender'].unique()) == {'f', 'm'}, 'Found ' + str(set(df['gender'].unique()))
    df['time'] = df['time'].apply(lambda s: pd.Timedelta(s) if s.lower() not in ['cot', 'dnf', 'dq'] else pd.NaT )
    df = df.reset_index(drop=True)
    return df

# Go to https://pickmyrace.id/preliminary-bali-ultra-trail-2024/, lalu save page as HTML (page only)
def read_but(filename: str, table_id: str) -> pd.DataFrame:
    return read_pickmyrace(filename, table_id)

# Jabar ultra: https://jabarultra.com/race-result-2024/, and save table to excel
def read_jbu(filename: str) -> pd.DataFrame:
    return read_scorenow(filename)

# UI Trail Race 2024: go to https://pickmyrace.id/preliminary-result-ui-trail-race-2024/ and save page
def read_ui_trail(filename: str, table_id: str) -> pd.DataFrame:
    return read_pickmyrace(filename, table_id)

## Daftar Race

In [99]:
%%time

@dataclass
class Race:
    distance: float
    eg: float
    load_fn: Callable
    quals: List[str] = field(default_factory=list)  # ITRA and UTMB qualifications
    dist_name: str = None    # name if distance is same for the same race
    peaks: List[str] = None  # Info about peaks, views
    ml: int = None           # ITRA mountain level
    fl: int = None           # ITRA finisher level
    nl: bool = None          # ITRA national league
    df: pd.DataFrame = None  # DataFrame
    dnf: int = None          # Number of DNFs
    times: pd.Series = None  # Finisher time
    mean: float = None       # in hours
    median: float = None     # in hours
    event = None             # Parent event

@dataclass
class Event:
    code: str
    title: str
    date: str
    instagram_handle: str
    url: str
    itra_id: int = None
    races: List[Race] = field(default_factory=list)

# Event codes
BRO = "Bromo Mar"
BTG = "Bromo Tgger"
BTR = "BTR"
BUT = "BUT"
CTC = "CTC"
DCR = "Dieng Caldera"
JUT = "JUT"
MAN = "Mantra"
MMD = "MMDT"
MSR = "MSR"
MST = "MesaStila100"
RIN = "Rinjani100"
SLU = "SLU"
SSC = "SSC"
UIT = "UI Trail"
VTM = "V. Telomoyo"

# ITRA and UTMB qualifications
ITRA0 = 'ITRA0'
ITRA1 = 'ITRA1'
ITRA2 = 'ITRA2'
ITRA3 = 'ITRA3'
ITRA4 = 'ITRA4'
ITRA5 = 'ITRA5'
ITRA6 = 'ITRA6'
UT20K =  'UTMB20K'
UT50K =  'UTMB50K'
UT100K = 'UTMB100K'
UT100M = 'UTMB100M'

events = [
    Event(BTR, 'Bali Trail Run', '2024-05-12', 'balitrailrunning', 'balitrailrunning.com', 89832, [
        Race(7, 300, lambda:read_btr('data/2024/btr7k.xlsx')),
        Race(15, 973, lambda:read_btr('data/2024/btr15k.xlsx'), [UT20K], peaks=['Batur']),
        Race(30, 1340, lambda:read_btr('data/2024/btr30k.xlsx'), [ITRA1, UT20K],
             peaks='Batur', ml=4, fl=230, nl=True),
        Race(55, 3778, lambda:read_btr('data/2024/btr55k.xlsx'), [ITRA3, UT50K],
             peaks=['Batur', 'Abang'], ml=9, fl=270, nl=True),
        Race(85, 5250, lambda:read_btr('data/2024/btr85k.xlsx'), [ITRA4, UT100K],
             peaks=['Batur', 'Abang'], ml=7, fl=320, nl=True),
    ]),
    Event(BUT, 'Bali Ultra Trail', '2024-08-03', 'baliultratrail.official', 'baliultratrail.com', 86937, [
        Race(12, 330, lambda:read_but('data/2024/but.html', 'table_1'), [ITRA0], ml=3, fl=170, nl=True),
        Race(25, 1650, lambda:read_but('data/2024/but.html', 'table_2'), [ITRA2, UT20K],
             peaks=['Batur'], ml=7, fl=190, nl=1, ),
        Race(50, 2730, lambda:read_but('data/2024/but.html', 'table_3'), [ITRA3, UT50K],
             peaks=['Batur2x'], ml=6, fl=270, nl=1, ),
        Race(80, 4400, lambda:read_but('data/2024/but.html', 'table_4'), [ITRA4, UT100K],
             peaks=['Batur2x'], fl=300, nl=1, ),
    ]),
    Event(BRO, 'Bromo Marathon', '2023-09-03', 'bromomarathon', 'bromomarathon.com', None, [
        Race(5, 241, lambda:read_bromar('data/2023/bromar.json', '5K')),
        Race(10, 426, lambda:read_bromar('data/2023/bromar.json', '10K')),
        Race(21, 977, lambda:read_bromar('data/2023/bromar.json', '21K'), peaks=['Bromo']),
        Race(42, 1930, lambda:read_bromar('data/2023/bromar.json', '42K'), peaks=['Bromo']),
    ]),
    Event(BTG, 'Bromo Tengger Trail Run', '2024-07-28', 'bromo.tenggertrailrun', '', 93006, [
        Race(11, 480, lambda:read_bromotengger('data/2024/bromo11k.csv')),
        Race(21, 1000, lambda:read_bromotengger('data/2024/bromo21k.csv'), [ITRA1, UT20K],
             peaks=['Bromo'], ml=5, fl=190, nl=True),
    ]),
    Event(CTC, 'Coast to Coast', '2024-02-25', 'ctc.ultra', 'ctcultra.com', 88662, [
        Race(5, 30, None, [], median=0.6),
        Race(15, 300, lambda:read_ctc('data/2024/ctc.html', 'table_1'), [ITRA0],
             ml=2, fl=160, nl=1),
        Race(30, 1040, lambda:read_ctc('data/2024/ctc.html', 'table_2'), [ITRA0, UT20K],
             ml=3, fl=250, nl=1),
        Race(50, 1620, lambda:read_ctc('data/2024/ctc.html', 'table_3'), [ITRA2, UT50K],
             ml=3, fl=290, nl=1),
        Race(80, 2550, lambda:read_ctc('data/2024/ctc.html', 'table_4'), [ITRA3, UT100K],
             ml=3, fl=370, nl=1),
    ]),
    Event(DCR, 'Dieng Caldera Race', '2024-06-09', 'diengcalderarace', 'diengcalderarace.com', 93604, [
        Race(10, 495, lambda:read_dcr('data/2024/dcr.html', 'table_1'), [ITRA0],
             ml=5, fl=130, nl=1),
        Race(21, 1185, lambda:read_dcr('data/2024/dcr.html', 'table_2'), [ITRA1, UT20K],
             ml=6, fl=190, nl=1),
        Race(42, 2630, lambda:read_dcr('data/2024/dcr.html', 'table_3'), [ITRA2, UT50K],
             ml=7, fl=260, nl=1),
        Race(75, 4850, lambda:read_dcr('data/2024/dcr.html', 'table_4'), [ITRA4, UT100K],
             ml=8, fl=290, nl=1),
    ]),
    Event(JUT, 'Jabar Ultra Trail', '2024-06-09', 'jabarultra', 'jabarultra.com', 92806, [
        Race(22, 2500, lambda:read_jbu('data/2024/jabarultra-22k.xlsx'), [ITRA2, UT20K],
             peaks=['Ciremai'], ml=12, fl=180, nl=1),
        Race(55, 6010, lambda:read_jbu('data/2024/jabarultra-55k.xlsx'), [ITRA3, UT100K],
             peaks=['Ciremai 3x'], ml=12, fl=220, nl=1),
    ]),
    Event(MAN, 'Mantra 116', '2024-07-07', 'mantra116.id', 'mantra116.com', 90032, [
        Race(10, 620, lambda:read_mantra('data/2024/mantra116-10k.json'), [ITRA0],
             ml=6, fl=140, nl=1),
        Race(17, 1000, lambda:read_mantra('data/2024/mantra116-17k.json'), [ITRA1, UT20K],
             ml=6, fl=220, nl=1),
        Race(34, 3050, lambda:read_mantra('data/2024/mantra116-34k.json'), [ITRA2, UT50K],
             dist_name='Arjuno', peaks=['Arjuno'], ml=12, fl=200, nl=1),
        Race(38, 2750, lambda:read_mantra('data/2024/mantra116-38k.json'), [ITRA2, UT50K],
             dist_name='Welirang', peaks=['Welirang'], ml=9, fl=220, nl=1),
        Race(68, 5000, lambda:read_mantra('data/2024/mantra116-68k.json'), [ITRA4, UT100K],
             peaks=['Arjuno', 'Welirang'], ml=10, fl=280, nl=1),
        Race(116, 7400, lambda:read_mantra('data/2024/mantra116-116k.json'), [ITRA5, UT100M],
             peaks=['Arjuno', 'Welirang'], ml=9, fl=380, nl=1),
    ]),
    Event(MMD, 'Merapi Merbabu De Trail', '2024-08-04', 'merapi_merbabu.detrail',
          'www.merapimerbabudetrail.com', 90917, [
        Race(5, 700, lambda:read_mmdt('data/2024/mmdt-5k.html'), [ITRA0],
             ml=11, fl=150, nl=1),
        Race(10, 1320, lambda:read_mmdt('data/2024/mmdt-10k.html'), [ITRA0, UT20K],
             peaks=['Merbabu'], ml=12, fl=140, nl=1),
        Race(20, 2940, lambda:read_mmdt('data/2024/mmdt-20k.html'), [ITRA2, UT20K],
             peaks=['Merbabu 2x'], ml=12, fl=140, nl=1),
    ]),
    Event(MSR, 'Merbabu Sky Race', '2024-04-28', 'merbabu_skyrace', 'merbabuskyrace.com', 84613, [
        Race(5, 170, lambda:read_msr('data/2024/msr5k.html'), []),
        Race(10, 810, lambda:read_msr('data/2024/msr10k.html'), [ITRA0],
            ml=9, fl=210, nl=1),
        Race(20, 1830, lambda:read_msr('data/2024/msr20k.html'), [ITRA1, UT20K],
             peaks=['Merbabu'], ml=12, nl=1),
        Race(40, 4290, lambda:read_msr('data/2024/msr40k.html'), [ITRA3, UT50K],
             peaks=['Merbabu 2x'], ml=12, nl=1),
        Race(50, 5970, lambda:read_msr('data/2024/msr50k.html'), [ITRA3, UT100K],
             peaks=['Merbabu 3x'], ml=12, fl=220, nl=1),
    ]),
    Event(MST, 'Mesastila 100', '2023-10-08', 'mesastila100', 'mesastila100.com', 82926, [
        Race(21, 1230, lambda:read_itra('data/2023/mesastila100-21k.xlsx'), [ITRA0], 
             ml=6, fl=220, nl=1),
    ]),
    Event(RIN, 'Rinjani 100', '2024-05-26', 'rinjani100.official', 'fonesport.id/rinjani100', 91507, [
        Race(27, 1847, lambda:read_rinjani100('data/2024/rinjani100-27k.xlsx'), [ITRA1, UT20K],
             ml=7, fl=210, nl=1),
        Race(36, 3179, lambda:read_rinjani100('data/2024/rinjani100-36k.xlsx'), [ITRA2, UT50K],
             peaks=['Rinjani'], ml=11, nl=1),
        Race(60, 5493, lambda:read_rinjani100('data/2024/rinjani100-60k.xlsx'), [ITRA3, UT100K],
             peaks=['Rinjani'], ml=12, fl=280, nl=1),
        Race(100, 9194, lambda:read_rinjani100('data/2024/rinjani100-100k.xlsx'), [ITRA5, UT100M],
             peaks=['Rinjani'], ml=12, fl=350, nl=1),
        Race(162, 13646, lambda:read_rinjani100('data/2024/rinjani100-162k.xlsx'), [ITRA6, UT100M],
             peaks=['Rinjani'], ml=12, nl=1),
    ]),
    Event(SLU, 'Siksorogo Lawu Ultra', '2023-12-03', 'siksorogolawuultra', 'siksorogo.id', 88372, [
        Race(7, 400, lambda:read_slu('data/2023/slu7k.json')),
        Race(15, 1200, lambda:read_slu('data/2023/slu15k.json'), [ITRA0, ],
             ml=6, nl=1),
        Race(30, 1800, lambda:read_slu('data/2023/slu30k.json'), [ITRA1, UT20K],
             ml=7, fl=210, nl=1),
        Race(50, 3800, lambda:read_slu('data/2023/slu50k.json'), [ITRA3, UT50K],
             peaks=['Lawu'], ml=8, fl=280, nl=1),
        Race(80, 5400, lambda:read_slu('data/2023/slu80k.json'), [ITRA4, UT100K],
             peaks=['Lawu'], ml=9, fl=290, nl=1),
    ]),
    Event(SSC, 'Sindoro Sumbing Challenge', '2024-05-05', 'sindoro_sumbing_challenge',
          'www.sindorosumbingchallenge.com/', 89387, [
        Race(20, 1963, lambda:read_itra('data/2024/ssc-sumbing.xlsx'), [ITRA1],
             dist_name='Sumbing', peaks=['Sumbing'], ml=12, fl=190, nl=1),
        Race(20, 2076, lambda:read_itra('data/2024/ssc-sindoro.xlsx'), [ITRA1],
             dist_name='Sindoro', peaks=['Sindoro'], ml=12, fl=190, nl=1),
        Race(35, 4046, lambda:read_itra('data/2024/ssc-40k.xlsx'), [ITRA3],
             peaks=['Sindoro', 'Sumbing'], ml=12, fl=180, nl=1),
    ]),
    Event(UIT, 'UI Trail Race', '2024-08-11', 'uitrailrace', 'uitrailrun.com', 94755, [
        Race(5, 180, lambda:read_ui_trail('data/2024/uitrailrace.html', 'table_1'),
             ml=3, fl=130, nl=1),
        Race(10, 650, lambda:read_ui_trail('data/2024/uitrailrace.html', 'table_2'), [ITRA0],
             ml=6, fl=130, nl=1),
        Race(20, 1500, lambda:read_ui_trail('data/2024/uitrailrace.html', 'table_3'),[ITRA1],
             ml=7, fl=150, nl=1),
        Race(40, 2400, lambda:read_ui_trail('data/2024/uitrailrace.html', 'table_4'), [ITRA2],
             ml=7, fl=220, nl=1),
        Race(80, 4800, lambda:read_ui_trail('data/2024/uitrailrace.html', 'table_5'), [ITRA4],
             ml=8, fl=320, nl=1),
    ]),
    Event(VTM, 'Vertical Telomoyo', '2023-10-01', 'vertical_telomoyo', 'verticaltelomoyo.com', 85282, [
        Race(7, 810, lambda:read_msr('data/2023/telomoyo7k.html'), [ITRA0],
             peaks=['Telomoyo'], ml=11, fl=190, nl=1),
        Race(27, 1420, lambda:read_msr('data/2023/telomoyo27k.html'), [ITRA1, UT20K],
             peaks=['Telomoyo', 'Andong'], ml=5, fl=230, nl=1, ),
    ]),
]


def load_races():
    for i, event in enumerate(events):
        for j, race in enumerate(event.races):
            print(f'Reading race data {i+1}/{len(events)} {event.code} {j+1}/{len(event.races)}..         \r', end='')
            race.event = event
            if race.load_fn is not None:
                race.df = race.load_fn()
                race.dnf = sum(pd.isnull(race.df['time']))
                race.times = race.df.loc[ race.df['time'].notnull(), 'time'].dt.total_seconds() / 3600
                race.mean = race.times.mean()
                race.median = race.times.median()
        
    print('\nDone')
    
load_races()

Reading race data 16/16 V. Telomoyo 2/2..          
Done
CPU times: user 20.7 s, sys: 44.5 ms, total: 20.8 s
Wall time: 20.7 s


## Plot Distribusi

In [150]:
def create_race_title(race: Race) -> str:
    dist_name = f'({race.dist_name}) ' if race.dist_name else ''
    title = f'{race.event.code} {race.distance:.0f}K {race.eg:.0f}m {dist_name}({race.event.date})'
    title = title + f' finishers: {len(race.times)}'
    if race.dnf:
        title += f', dnf: {race.dnf} ({race.dnf/(race.dnf+len(race.times)):.0%})'
    return title

def plot_distribution(race: Race, names: List[str] = [], ax=None):
    bins = 30
    df = race.df
    times = df.loc[ df['time'].notnull(), 'time'].dt.total_seconds() / 3600
    
    if ax:
        ax.hist(times, bins=bins, alpha=0.6)
        ax.grid()
        show = False
    else:
        ax = times.hist(bins=bins, alpha=0.6, figsize=(8,5))
        show = True
    ax.set_xlabel('Finished Time (Hour)')
    ax.set_ylabel('Number of runners')
    
    ax.set_title(create_race_title(race))
    ax.axvline(x=times.mean(), linestyle=':',
              color='k', alpha=1, zorder=100, label='mean')
    ax.axvline(x=times.median(), linestyle='--',
              color='k', alpha=1, zorder=100, label='median')
    
    if race.quals:
        quals = ', '.join(race.quals)
        right = ax.get_xlim()[1]
        top = ax.get_ylim()[1]
        ax.text(right, top*0.99, quals,
                horizontalalignment='right',
                verticalalignment='top', fontweight='book')
    
    icolor = 0
    for name in names:
        found = df[ df['name'].str.contains(name) ]
        if not len(found):
            continue

        a = found.iloc[0]
        t = a['time'].total_seconds() / 3600
        ax.axvline(x=t, color=f'C{1+icolor}', alpha=1, zorder=10, label=name)
        icolor += 1
            
    
    ax.legend(loc='upper left')
    if show:
        plt.show()
        
def plot_age_distribution(ages: pd.Series):
    ages = ages[(ages>10) & (ages<100)]
    ax = ages.hist(bins=20, figsize=(8,5))
    ax.set_title('Age Distribution' + f' (finishers: {len(ages)})')


## Generate README

In [171]:
#%%time

from urllib.parse import urljoin
import matplotlib.image as pltimg
import os

readme = '''# Statistik Event Trail/Ultra Running Indonesia

Berikut distribusi finisher dari race-race yang terdata, diurutkan bds
median finish time, agar bisa dikira-kira tingkat kesulitan dari race itu.

Tapii... harap diwaspadai, distribusi hanya menghitung finish time dari finisher.
Harap diperhatikan juga DNF ratenya. Kalau median finish time lebih rendah tapi
DNF rate lebih tinggi, kemungkinan racenya lebih berat (misalnya race2 MSR).

Beberapa event juga tidak memberikan data peserta yang over COT atau DNF (misalnya
CTC). Dari bentuk distribusinya, kalau puncaknya di kanan (left skewed, misalnya
CTC 30K, 50K) maka kemungkinan banyak peserta yg DNF/over COT.

Untuk tiap race juga ditampilkan kualifikasi ITRA and ITMB, ITRA mountain level
dan ITRA finisher level, dan finish time saya dan bbrp teman yg saya tahu dan
selebriti (namanya engga disebut lengkap) biar mantau posisi aja hehe.

Event-event yg terdata:

{list_events}

Enjoy dan fork/PR ya.
'''


list_nama = ['nny Prij', 'odong Cah', 'lfin Bah', 'griawan Su', 'iza Satr', 'ranindo', 'ommy Bas',
             'aldy Mas', 'eny Roh', 'gus Mahap', 'endi Dw', 'ad Suhu', 'ad Hariy',
             'ani Chi']

def create_img_link(img: str, url: str, method=0) -> str:
    if method==0:
        return f'[![img]({img})]({url})'
    elif method==1:
        return f'[<img src="{img}">]({url}) '
    elif method==2:
        return f'<a href="{url}">![Foo]({img})</a>'
    elif method==3:
        return f'<a href="{url}"><img src="{img}"></a>'
    elif method==4:
        return f'[![image]({img} "icon")]({url})'
    else:
        assert False
    
def get_event_links(e: Event, which: str = 'web instagram itra') -> str:
    links = []
    if e.url and 'web' in which:
        url = ('https://' + e.url) if 'http' not in e.url else url
        #links += create_img_link('images/website_icon.png', url)
        links.append(f'[homepage]({url})')
    if e.instagram_handle and 'insta' in which:
        url = f'https://www.instagram.com/{e.instagram_handle}/'
        #links += create_img_link('images/instagram_icon.jpg', url)
        links.append(f' [instagram]({url})')
    if e.itra_id and 'itra' in which:
        url = f'https://itra.run/Races/RaceDetails/{e.itra_id}/'
        #links += create_img_link('images/itra_icon.png', url)
        links.append(f'[ITRA]({url})')
    return ' | '.join(links)

def create_race_table(races: List[Race]) -> str:
    md = ''
    md += '| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |\n'
    md += '|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|\n'
    for race in races:
        title = f'[{race.event.code}](https://www.instagram.com/{race.event.instagram_handle}/)'
        dnf = str(race.dnf) if race.dnf else '   '
        dnf_rate = f'({race.dnf/(race.dnf+len(race.times)):.0%})' if race.dnf else ''
        n_finishers = len(race.times) if race.times is not None else ''
        med = f'{race.median:.1f}' if race.median else ''
        max = f'{race.times.max():.1f}' if race.times is not None else '-'
        quals = ''
        for qual in race.quals:
            quals += f' <img src="images/{qual}.png" height="16">'
        md += f'| {title} | {race.distance:.0f} | {race.eg:.0f} | {n_finishers} | {med} / {max} | {dnf} {dnf_rate} | {quals} | {race.ml or ""} | {race.fl or ""} |\n'
    return md
    

def generate_readme():
    md = '' + readme
    
    list_events = ''
    races = []
    for ie, e in enumerate(events):
        list_events += f'- **{e.title}** ({e.code}): tanggal {e.date}, {len(e.races)} races (links: {get_event_links(e)})\n'
        races.extend( e.races )
    
    md = md.replace('{list_events}', list_events)
    
    hours = [2, 4, 6, 8, 10, 12, 16, 24, 100]
    ncols = 2

    prev_h = 0
    for h in hours:
        print(f'Generating {prev_h}-{h}h..   \r', end='')
        # For the table, list all races even without finisher data
        lst = [ r for r in races if r.median>=prev_h and r.median<h ]
        if not lst:
            continue
        lst = sorted(lst, key=lambda r: r.median)

        fname = f'images/{prev_h}-{h}h.png'
        md += f'## {prev_h} - {h} jam\n\n'
        md += create_race_table(lst)
        md += f'\n![stat]({fname} "Statistik")\n\n'
        
        # List again only for races with finisher data
        lst = [ r for r in races if r.median>=prev_h and r.median<h and r.df is not None]
        lst = sorted(lst, key=lambda r: r.median)
        nrows = (len(lst)+ncols-1)//ncols
        
        fig, axs = plt.subplots(nrows, ncols, figsize=(ncols*6.5, nrows*4), facecolor = 'aliceblue')
        r,c=0,0
        for i in range(len(lst)):
            race = lst[i]
            if race.df is None:
                continue
            plot_distribution(race, list_nama, ax=axs[r][c])
            c+=1
            if c==ncols:
                r+=1
                c=0

        if c==1:
            fig.delaxes(axs[r][c])
            
        fig.tight_layout()
        
        plt.savefig(fname)
        plt.close(fig)

        
        prev_h = h

    md += '\n\n(Catatan: file ini dihasilkan oleh kode di .ipynb)\n'
    with open('README.md', 'wt') as f:
        f.write(md)
        
    print('\nDone.')
        
generate_readme()

Generating 24-100h..   
Done.


In [172]:
with open('README.md') as f:
    display(Markdown(f.read()))

# Statistik Event Trail/Ultra Running Indonesia

Berikut distribusi finisher dari race-race yang terdata, diurutkan bds
median finish time, agar bisa dikira-kira tingkat kesulitan dari race itu.

Tapii... harap diwaspadai, distribusi hanya menghitung finish time dari finisher.
Harap diperhatikan juga DNF ratenya. Kalau median finish time lebih rendah tapi
DNF rate lebih tinggi, kemungkinan racenya lebih berat (misalnya race2 MSR).

Beberapa event juga tidak memberikan data peserta yang over COT atau DNF (misalnya
CTC). Dari bentuk distribusinya, kalau puncaknya di kanan (left skewed, misalnya
CTC 30K, 50K) maka kemungkinan banyak peserta yg DNF/over COT.

Untuk tiap race juga ditampilkan kualifikasi ITRA and ITMB, ITRA mountain level
dan ITRA finisher level, dan finish time saya dan bbrp teman yg saya tahu dan
selebriti (namanya engga disebut lengkap) biar mantau posisi aja hehe.

Event-event yg terdata:

- **Bali Trail Run** (BTR): tanggal 2024-05-12, 5 races (links: [homepage](https://balitrailrunning.com) |  [instagram](https://www.instagram.com/balitrailrunning/) | [ITRA](https://itra.run/Races/RaceDetails/89832/))
- **Bali Ultra Trail** (BUT): tanggal 2024-08-03, 4 races (links: [homepage](https://baliultratrail.com) |  [instagram](https://www.instagram.com/baliultratrail.official/) | [ITRA](https://itra.run/Races/RaceDetails/86937/))
- **Bromo Marathon** (Bromo Mar): tanggal 2023-09-03, 4 races (links: [homepage](https://bromomarathon.com) |  [instagram](https://www.instagram.com/bromomarathon/))
- **Bromo Tengger Trail Run** (Bromo Tgger): tanggal 2024-07-28, 2 races (links:  [instagram](https://www.instagram.com/bromo.tenggertrailrun/) | [ITRA](https://itra.run/Races/RaceDetails/93006/))
- **Coast to Coast** (CTC): tanggal 2024-02-25, 5 races (links: [homepage](https://ctcultra.com) |  [instagram](https://www.instagram.com/ctc.ultra/) | [ITRA](https://itra.run/Races/RaceDetails/88662/))
- **Dieng Caldera Race** (Dieng Caldera): tanggal 2024-06-09, 4 races (links: [homepage](https://diengcalderarace.com) |  [instagram](https://www.instagram.com/diengcalderarace/) | [ITRA](https://itra.run/Races/RaceDetails/93604/))
- **Jabar Ultra Trail** (JUT): tanggal 2024-06-09, 2 races (links: [homepage](https://jabarultra.com) |  [instagram](https://www.instagram.com/jabarultra/) | [ITRA](https://itra.run/Races/RaceDetails/92806/))
- **Mantra 116** (Mantra): tanggal 2024-07-07, 6 races (links: [homepage](https://mantra116.com) |  [instagram](https://www.instagram.com/mantra116.id/) | [ITRA](https://itra.run/Races/RaceDetails/90032/))
- **Merapi Merbabu De Trail** (MMDT): tanggal 2024-08-04, 3 races (links: [homepage](https://www.merapimerbabudetrail.com) |  [instagram](https://www.instagram.com/merapi_merbabu.detrail/) | [ITRA](https://itra.run/Races/RaceDetails/90917/))
- **Merbabu Sky Race** (MSR): tanggal 2024-04-28, 5 races (links: [homepage](https://merbabuskyrace.com) |  [instagram](https://www.instagram.com/merbabu_skyrace/) | [ITRA](https://itra.run/Races/RaceDetails/84613/))
- **Mesastila 100** (MesaStila100): tanggal 2023-10-08, 1 races (links: [homepage](https://mesastila100.com) |  [instagram](https://www.instagram.com/mesastila100/) | [ITRA](https://itra.run/Races/RaceDetails/82926/))
- **Rinjani 100** (Rinjani100): tanggal 2024-05-26, 5 races (links: [homepage](https://fonesport.id/rinjani100) |  [instagram](https://www.instagram.com/rinjani100.official/) | [ITRA](https://itra.run/Races/RaceDetails/91507/))
- **Siksorogo Lawu Ultra** (SLU): tanggal 2023-12-03, 5 races (links: [homepage](https://siksorogo.id) |  [instagram](https://www.instagram.com/siksorogolawuultra/) | [ITRA](https://itra.run/Races/RaceDetails/88372/))
- **Sindoro Sumbing Challenge** (SSC): tanggal 2024-05-05, 3 races (links: [homepage](https://www.sindorosumbingchallenge.com/) |  [instagram](https://www.instagram.com/sindoro_sumbing_challenge/) | [ITRA](https://itra.run/Races/RaceDetails/89387/))
- **UI Trail Race** (UI Trail): tanggal 2024-08-11, 5 races (links: [homepage](https://uitrailrun.com) |  [instagram](https://www.instagram.com/uitrailrace/) | [ITRA](https://itra.run/Races/RaceDetails/94755/))
- **Vertical Telomoyo** (V. Telomoyo): tanggal 2023-10-01, 2 races (links: [homepage](https://verticaltelomoyo.com) |  [instagram](https://www.instagram.com/vertical_telomoyo/) | [ITRA](https://itra.run/Races/RaceDetails/85282/))


Enjoy dan fork/PR ya.
## 0 - 2 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [CTC](https://www.instagram.com/ctc.ultra/) | 5 | 30 |  | 0.6 / - |      |  |  |  |
| [Bromo Mar](https://www.instagram.com/bromomarathon/) | 5 | 241 | 206 | 0.9 / 6.0 | 1 (0%) |  |  |  |
| [MSR](https://www.instagram.com/merbabu_skyrace/) | 5 | 170 | 207 | 1.0 / 1.7 |      |  |  |  |
| [V. Telomoyo](https://www.instagram.com/vertical_telomoyo/) | 7 | 810 | 81 | 1.2 / 2.3 | 5 (6%) |  <img src="images/ITRA0.png" height="16"> | 11 | 190 |
| [BTR](https://www.instagram.com/balitrailrunning/) | 7 | 300 | 311 | 1.2 / 2.7 |      |  |  |  |
| [UI Trail](https://www.instagram.com/uitrailrace/) | 5 | 180 | 208 | 1.2 / 2.4 | 1 (0%) |  | 3 | 130 |
| [Bromo Mar](https://www.instagram.com/bromomarathon/) | 10 | 426 | 192 | 1.7 / 2.7 | 5 (3%) |  |  |  |
| [SLU](https://www.instagram.com/siksorogolawuultra/) | 7 | 400 | 723 | 1.7 / 3.9 |      |  |  |  |
| [Dieng Caldera](https://www.instagram.com/diengcalderarace/) | 10 | 495 | 246 | 1.9 / 2.8 |      |  <img src="images/ITRA0.png" height="16"> | 5 | 130 |

![stat](images/0-2h.png "Statistik")

## 2 - 4 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [BUT](https://www.instagram.com/baliultratrail.official/) | 12 | 330 | 70 | 2.3 / 3.6 |      |  <img src="images/ITRA0.png" height="16"> | 3 | 170 |
| [Mantra](https://www.instagram.com/mantra116.id/) | 10 | 620 | 311 | 2.4 / 4.5 | 59 (16%) |  <img src="images/ITRA0.png" height="16"> | 6 | 140 |
| [Bromo Tgger](https://www.instagram.com/bromo.tenggertrailrun/) | 11 | 480 | 157 | 2.5 / 4.3 |      |  |  |  |
| [MMDT](https://www.instagram.com/merapi_merbabu.detrail/) | 5 | 700 | 97 | 2.5 / 3.5 | 51 (34%) |  <img src="images/ITRA0.png" height="16"> | 11 | 150 |
| [CTC](https://www.instagram.com/ctc.ultra/) | 15 | 300 | 1100 | 3.1 / 4.0 |      |  <img src="images/ITRA0.png" height="16"> | 2 | 160 |
| [MSR](https://www.instagram.com/merbabu_skyrace/) | 10 | 810 | 295 | 3.3 / 4.0 | 98 (25%) |  <img src="images/ITRA0.png" height="16"> | 9 | 210 |
| [SLU](https://www.instagram.com/siksorogolawuultra/) | 15 | 1200 | 932 | 3.5 / 5.9 |      |  <img src="images/ITRA0.png" height="16"> | 6 |  |
| [Bromo Mar](https://www.instagram.com/bromomarathon/) | 21 | 977 | 153 | 3.8 / 6.8 | 4 (3%) |  |  |  |
| [UI Trail](https://www.instagram.com/uitrailrace/) | 10 | 650 | 210 | 3.8 / 7.3 | 2 (1%) |  <img src="images/ITRA0.png" height="16"> | 6 | 130 |
| [Mantra](https://www.instagram.com/mantra116.id/) | 17 | 1000 | 265 | 3.9 / 7.6 | 49 (16%) |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 6 | 220 |

![stat](images/2-4h.png "Statistik")

## 4 - 6 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [Bromo Tgger](https://www.instagram.com/bromo.tenggertrailrun/) | 21 | 1000 | 76 | 4.2 / 6.8 | 9 (11%) |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 5 | 190 |
| [BTR](https://www.instagram.com/balitrailrunning/) | 15 | 973 | 681 | 4.5 / 7.0 | 22 (3%) |  <img src="images/UTMB20K.png" height="16"> |  |  |
| [MMDT](https://www.instagram.com/merapi_merbabu.detrail/) | 10 | 1320 | 166 | 5.3 / 6.9 | 26 (14%) |  <img src="images/ITRA0.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 12 | 140 |
| [V. Telomoyo](https://www.instagram.com/vertical_telomoyo/) | 27 | 1420 | 42 | 5.4 / 7.6 | 4 (9%) |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 5 | 230 |
| [MesaStila100](https://www.instagram.com/mesastila100/) | 21 | 1230 | 72 | 5.4 / 8.6 | 1 (1%) |  <img src="images/ITRA0.png" height="16"> | 6 | 220 |
| [Dieng Caldera](https://www.instagram.com/diengcalderarace/) | 21 | 1185 | 268 | 5.9 / 8.7 |      |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 6 | 190 |

![stat](images/4-6h.png "Statistik")

## 6 - 8 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [BTR](https://www.instagram.com/balitrailrunning/) | 30 | 1340 | 302 | 6.1 / 9.0 | 10 (3%) |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 4 | 230 |
| [CTC](https://www.instagram.com/ctc.ultra/) | 30 | 1040 | 245 | 6.5 / 7.0 |      |  <img src="images/ITRA0.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 3 | 250 |
| [UI Trail](https://www.instagram.com/uitrailrace/) | 20 | 1500 | 150 | 6.9 / 10.1 | 11 (7%) |  <img src="images/ITRA1.png" height="16"> | 7 | 150 |
| [Bromo Mar](https://www.instagram.com/bromomarathon/) | 42 | 1930 | 38 | 7.0 / 8.5 | 9 (19%) |  |  |  |
| [SSC](https://www.instagram.com/sindoro_sumbing_challenge/) | 20 | 2076 | 65 | 7.4 / 9.0 | 5 (7%) |  <img src="images/ITRA1.png" height="16"> | 12 | 190 |
| [BUT](https://www.instagram.com/baliultratrail.official/) | 25 | 1650 | 111 | 7.5 / 12.2 | 24 (18%) |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 7 | 190 |
| [SLU](https://www.instagram.com/siksorogolawuultra/) | 30 | 1800 | 645 | 7.6 / 10.0 |      |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 7 | 210 |
| [MSR](https://www.instagram.com/merbabu_skyrace/) | 20 | 1830 | 532 | 7.6 / 9.0 | 149 (22%) |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 12 |  |
| [SSC](https://www.instagram.com/sindoro_sumbing_challenge/) | 20 | 1963 | 76 | 7.7 / 8.9 | 4 (5%) |  <img src="images/ITRA1.png" height="16"> | 12 | 190 |
| [JUT](https://www.instagram.com/jabarultra/) | 22 | 2500 | 101 | 7.8 / 10.0 | 39 (28%) |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 12 | 180 |
| [Rinjani100](https://www.instagram.com/rinjani100.official/) | 27 | 1847 | 75 | 7.8 / 9.0 | 21 (22%) |  <img src="images/ITRA1.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 7 | 210 |

![stat](images/6-8h.png "Statistik")

## 8 - 12 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [Dieng Caldera](https://www.instagram.com/diengcalderarace/) | 42 | 2630 | 106 | 10.5 / 14.0 |      |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 7 | 260 |
| [CTC](https://www.instagram.com/ctc.ultra/) | 50 | 1620 | 178 | 11.0 / 12.0 |      |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 3 | 290 |
| [UI Trail](https://www.instagram.com/uitrailrace/) | 40 | 2400 | 38 | 11.8 / 15.2 | 11 (22%) |  <img src="images/ITRA2.png" height="16"> | 7 | 220 |
| [BUT](https://www.instagram.com/baliultratrail.official/) | 50 | 2730 | 70 | 11.8 / 15.7 | 16 (19%) |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 6 | 270 |
| [MMDT](https://www.instagram.com/merapi_merbabu.detrail/) | 20 | 2940 | 83 | 12.0 / 13.9 | 13 (14%) |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB20K.png" height="16"> | 12 | 140 |

![stat](images/8-12h.png "Statistik")

## 12 - 16 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [Mantra](https://www.instagram.com/mantra116.id/) | 38 | 2750 | 184 | 12.5 / 15.0 | 54 (23%) |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 9 | 220 |
| [SSC](https://www.instagram.com/sindoro_sumbing_challenge/) | 35 | 4046 | 47 | 12.7 / 18.4 | 11 (19%) |  <img src="images/ITRA3.png" height="16"> | 12 | 180 |
| [Rinjani100](https://www.instagram.com/rinjani100.official/) | 36 | 3179 | 322 | 12.8 / 15.0 | 118 (27%) |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 11 |  |
| [Mantra](https://www.instagram.com/mantra116.id/) | 34 | 3050 | 193 | 12.9 / 16.2 | 96 (33%) |  <img src="images/ITRA2.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 12 | 200 |
| [SLU](https://www.instagram.com/siksorogolawuultra/) | 50 | 3800 | 231 | 14.2 / 16.8 |      |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 8 | 280 |
| [BTR](https://www.instagram.com/balitrailrunning/) | 55 | 3778 | 107 | 15.5 / 18.0 | 24 (18%) |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 9 | 270 |
| [CTC](https://www.instagram.com/ctc.ultra/) | 80 | 2550 | 64 | 15.9 / 18.0 |      |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 3 | 370 |

![stat](images/12-16h.png "Statistik")

## 16 - 24 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [MSR](https://www.instagram.com/merbabu_skyrace/) | 40 | 4290 | 42 | 16.2 / 17.8 | 63 (60%) |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB50K.png" height="16"> | 12 |  |
| [Dieng Caldera](https://www.instagram.com/diengcalderarace/) | 75 | 4850 | 30 | 17.2 / 23.6 |      |  <img src="images/ITRA4.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 8 | 290 |
| [BUT](https://www.instagram.com/baliultratrail.official/) | 80 | 4400 | 32 | 17.9 / 22.9 | 14 (30%) |  <img src="images/ITRA4.png" height="16"> <img src="images/UTMB100K.png" height="16"> |  | 300 |
| [Rinjani100](https://www.instagram.com/rinjani100.official/) | 60 | 5493 | 43 | 19.1 / 20.0 | 118 (73%) |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 12 | 280 |
| [MSR](https://www.instagram.com/merbabu_skyrace/) | 50 | 5970 | 24 | 19.5 / 23.6 | 24 (50%) |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 12 | 220 |
| [SLU](https://www.instagram.com/siksorogolawuultra/) | 80 | 5400 | 73 | 19.5 / 22.9 |      |  <img src="images/ITRA4.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 9 | 290 |
| [Mantra](https://www.instagram.com/mantra116.id/) | 68 | 5000 | 104 | 19.5 / 22.1 | 80 (43%) |  <img src="images/ITRA4.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 10 | 280 |
| [BTR](https://www.instagram.com/balitrailrunning/) | 85 | 5250 | 30 | 20.4 / 25.0 | 7 (19%) |  <img src="images/ITRA4.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 7 | 320 |
| [JUT](https://www.instagram.com/jabarultra/) | 55 | 6010 | 7 | 21.2 / 23.4 | 12 (63%) |  <img src="images/ITRA3.png" height="16"> <img src="images/UTMB100K.png" height="16"> | 12 | 220 |
| [UI Trail](https://www.instagram.com/uitrailrace/) | 80 | 4800 | 11 | 21.2 / 23.7 | 18 (62%) |  <img src="images/ITRA4.png" height="16"> | 8 | 320 |

![stat](images/16-24h.png "Statistik")

## 24 - 100 jam

| Event | Jarak | Eg (m) | Finishers | Med / Max (jam) | DNF (Rate) | Kualif. | Mtn Lvl | Fns Lvl |
|-------|-------|--------|-----------|-----------------|------------|---------|---------|---------|
| [Mantra](https://www.instagram.com/mantra116.id/) | 116 | 7400 | 59 | 29.8 / 33.3 | 51 (46%) |  <img src="images/ITRA5.png" height="16"> <img src="images/UTMB100M.png" height="16"> | 9 | 380 |
| [Rinjani100](https://www.instagram.com/rinjani100.official/) | 100 | 9194 | 13 | 33.8 / 35.4 | 71 (85%) |  <img src="images/ITRA5.png" height="16"> <img src="images/UTMB100M.png" height="16"> | 12 | 350 |
| [Rinjani100](https://www.instagram.com/rinjani100.official/) | 162 | 13646 | 6 | 52.0 / 54.4 | 27 (82%) |  <img src="images/ITRA6.png" height="16"> <img src="images/UTMB100M.png" height="16"> | 12 |  |

![stat](images/24-100h.png "Statistik")



(Catatan: file ini dihasilkan oleh kode di .ipynb)
